## Camadas Convolucionais

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input

def convlayer(hidden_ch=128,
              strides=[(1,1),(1,1)],
              norm='layer_norm2d'):
    conv2d_1x3 = tf.keras.layers.Conv2D(hidden_ch,
                                        kernel_size=(1, 3),
                                        strides=strides[0],
                                        padding='SAME',
                                        dilation_rate=(1, 1),
                                        kernel_initializer='glorot_uniform',
                                        bias_initializer='zeros')
    conv2d_3x1 = tf.keras.layers.Conv2D(hidden_ch,
                                        kernel_size=(3, 1),
                                        strides=strides[1],
                                        padding='SAME',
                                        dilation_rate=(1, 1),
                                        kernel_initializer='glorot_uniform',
                                        bias_initializer='zeros')
    if norm == 'layer_norm1d':
        BN_1x3 = tf.keras.layers.LayerNormalization(axis=-1)
        BN_3x1 = tf.keras.layers.LayerNormalization(axis=-1)
    elif norm == 'layer_norm2d':
        BN_1x3 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
        BN_3x1 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
    else:
        BN_1x3 = tf.keras.layers.BatchNormalization(axis=-1)
        BN_3x1 = tf.keras.layers.BatchNormalization(axis=-1)
        
    forward = tf.keras.Sequential([conv2d_1x3,
                                   tf.keras.layers.ELU(),
                                   BN_1x3,
                                   conv2d_3x1,
                                   tf.keras.layers.ELU(),
                                   BN_3x1
                                   ])
    
    return forward

2024-05-13 14:37:13.400090: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def create_sequential_front_conv(input_shape=(256,32,1),
                                 emb_sz=128,
                                 front_hidden_ch=[128, 128, 256, 256, 512, 512, 1024, 1024],
                                 front_strides=[[(1,2), (2,1)], [(1,2), (2,1)],
                                                [(1,2), (2,1)], [(1,2), (2,1)],
                                                [(1,1), (2,1)], [(1,2), (2,1)],
                                                [(1,1), (2,1)], [(1,2), (2,1)]],
                                 norm='layer_norm2d'):
    front_conv = tf.keras.Sequential(name='ConvLayers')
    if ((front_hidden_ch[-1] % emb_sz) != 0):
        front_hidden_ch[-1] = ((front_hidden_ch[-1]//emb_sz) + 1) * emb_sz

    for i in range(len(front_strides)):
        front_conv.add(convlayer(hidden_ch=front_hidden_ch[i], strides=front_strides[i], norm=norm))
    front_conv.add(tf.keras.layers.Flatten())

    return front_conv

## conv + densas + concat + normalize

In [3]:
def auxiliar(input1):#meter a create_sequential aqui dentro, senao era global
    conv_layer = create_sequential_front_conv(input_shape=(256,32,1),
                                               emb_sz=128,
                                               front_hidden_ch=[128, 128, 256, 256, 512, 512, 1024, 1024],
                                               front_strides=[[(1,2), (2,1)], [(1,2), (2,1)],
                                                              [(1,2), (2,1)], [(1,2), (2,1)],
                                                              [(1,1), (2,1)], [(1,2), (2,1)],
                                                              [(1,1), (2,1)], [(1,2), (2,1)]],
                                               norm='layer_norm2d')

    unit_dim = [32, 1]
    q = 128
    arquiteturas_densas = tf.keras.Sequential([tf.keras.layers.Dense(unit_dim[0], activation='elu'),
                                               tf.keras.layers.Dense(unit_dim[1])])

    x = input1
    #x reshape
    x = conv_layer(x)

    y_list = [0] * q
    x_split = tf.split(x, num_or_size_splits=128, axis=1)

    for v, k in enumerate(x_split):
        y_list[v] = arquiteturas_densas(k)

    out = tf.concat(y_list, axis=1)
    output = tf.math.l2_normalize(out, axis=1)
    return output

## Função do modelo para enviar para o run_train.ipynb

In [5]:
def get_fingerprinting(input1):
    output = auxiliar(input1)
    fingerprinting_model = Model(inputs=input1, outputs=output)
    return fingerprinting_model

## Teste

In [6]:
input1 = tf.keras.Input(shape=(256, 32, 1))

modelo = get_fingerprinting(input1)

In [10]:
modelo.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.FalseNegatives(),
    ],
)

In [11]:
import os
import sys
import yaml
from dataset import Dataset


def load_config(config_fname):
    config_filepath = '../../config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg


cfg=load_config('default')
dataset = Dataset(cfg)

ModuleNotFoundError: No module named 'dataset'

In [ ]:
modelo.fit(train_ds, epochs=1)

In [12]:
modelo.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 32, 1)]         0         []                            
                                                                                                  
 ConvLayers (Sequential)     (None, 1024)                 1689792   ['input_2[0][0]']             
                                                          0                                       
                                                                                                  
 tf.split (TFOpLambda)       [(None, 8),                  0         ['ConvLayers[0][0]']          
                              (None, 8),                                                          
                              (None, 8),                                                      

# **Rascunhos**

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input

2024-05-11 18:05:11.407951: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
input1 = Input(shape=(60,128))
input_shape = (1, 256, 32, 1)  # (batch_size, height, width, channels)

# Criando um tensor de entrada aleatório
input1 = np.random.rand(*input_shape)



input1 = tf.keras.Input(shape=(256, 32, 1))

In [4]:
#input1 = tf.ones(shape=(120, 256, 32, 1))

In [5]:
def convlayer(hidden_ch=128,
              strides=[(1,1),(1,1)],
              norm='layer_norm2d'):
    
    
    conv2d_1x3 = tf.keras.layers.Conv2D(hidden_ch,
                                        kernel_size=(1, 3),
                                        strides=strides[0],
                                        padding='SAME',
                                        dilation_rate=(1, 1),
                                        kernel_initializer='glorot_uniform',
                                        bias_initializer='zeros')
    conv2d_3x1 = tf.keras.layers.Conv2D(hidden_ch,
                                        kernel_size=(3, 1),
                                        strides=strides[1],
                                        padding='SAME',
                                        dilation_rate=(1, 1),
                                        kernel_initializer='glorot_uniform',
                                        bias_initializer='zeros')
    
    if norm == 'layer_norm1d':
        BN_1x3 = tf.keras.layers.LayerNormalization(axis=-1)
        BN_3x1 = tf.keras.layers.LayerNormalization(axis=-1)
    elif norm == 'layer_norm2d':
        BN_1x3 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
        BN_3x1 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
    else:
        BN_1x3 = tf.keras.layers.BatchNormalization(axis=-1) # Fix axis: 2020 Apr20
        BN_3x1 = tf.keras.layers.BatchNormalization(axis=-1)
        
    forward = tf.keras.Sequential([conv2d_1x3,
                                    tf.keras.layers.ELU(),
                                    BN_1x3,
                                    conv2d_3x1,
                                    tf.keras.layers.ELU(),
                                    BN_3x1
                                    ])
    
    return forward


def create_sequential_front_conv(input_shape=(256,32,1),
                                    emb_sz=128, # q
                                    front_hidden_ch=[128, 128, 256, 256, 512, 512, 1024, 1024],
                                    front_strides=[[(1,2), (2,1)], [(1,2), (2,1)],
                                                   [(1,2), (2,1)], [(1,2), (2,1)],
                                                   [(1,1), (2,1)], [(1,2), (2,1)],
                                                   [(1,1), (2,1)], [(1,2), (2,1)]],
                                    norm='layer_norm2d'):

    front_conv = tf.keras.Sequential(name='ConvLayers')
    if ((front_hidden_ch[-1] % emb_sz) != 0):
        front_hidden_ch[-1] = ((front_hidden_ch[-1]//emb_sz) + 1) * emb_sz

    for i in range(len(front_strides)):
        front_conv.add(convlayer(hidden_ch=front_hidden_ch[i], strides=front_strides[i], norm=norm))
    front_conv.add(tf.keras.layers.Flatten())

    return front_conv

In [6]:
conv_layer = create_sequential_front_conv(input_shape=(256,32,1),
                                    emb_sz=128, # q
                                    front_hidden_ch=[128, 128, 256, 256, 512, 512, 1024, 1024],
                                    front_strides=[[(1,2), (2,1)], [(1,2), (2,1)],
                                                   [(1,2), (2,1)], [(1,2), (2,1)],
                                                   [(1,1), (2,1)], [(1,2), (2,1)],
                                                   [(1,1), (2,1)], [(1,2), (2,1)]],
                                    norm='layer_norm2d')

unit_dim=[32, 1]
q=128
arquiteturas_densas = tf.keras.Sequential([tf.keras.layers.Dense(unit_dim[0], activation='elu'),
                                        tf.keras.layers.Dense(unit_dim[1])])

In [7]:
#input = tf.ones(shape=(120, 256, 32, 1))

In [8]:
#print(input1.shape[1])

In [9]:
#x = conv_layer(input1)
#[x.shape[0], q, -1]

In [10]:
x = conv_layer(input1)

y_list = [0]*q # lista vazia de tamanho 128, para guardar as 128 redes

#input = tf.ones((120, 1, 1, 1024))
#x = tf.reshape(x, shape=[input1.shape[0], q, -1])
#x = tf.reshape(x, shape=[x.shape[0], q, -1])

x_split = tf.split(x, num_or_size_splits=128, axis=1) # faz o split em 128 vetores de igual tamanho (8)

#Dados as 128 redes
for v, k in enumerate(x_split):
    y_list[v] = arquiteturas_densas(k)

#out = tf.keras.layers.Concatenate(y_list)
out = tf.concat(y_list, axis=1)
output = tf.math.l2_normalize(out, axis=1)
#return out

In [11]:
functional_model = Model(inputs=input1, outputs=output)

In [12]:
functional_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 32, 1)]         0         []                            
                                                                                                  
 ConvLayers (Sequential)     (None, 1024)                 1689792   ['input_2[0][0]']             
                                                          0                                       
                                                                                                  
 tf.split (TFOpLambda)       [(None, 8),                  0         ['ConvLayers[0][0]']          
                              (None, 8),                                                          
                              (None, 8),                                                      

In [13]:
def get_fingerprinting(input1, output):
    
    fingerprinting_model = tf.keras.Sequential(name='Fingerprinting')
    fingerprinting_model = Model(inputs=input1, outputs=output)
    
    return fingerprinting_model

In [15]:
#get_fingerprinting(input1, output)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 32, 1)]         0         []                            
                                                                                                  
 ConvLayers (Sequential)     (None, 1024)                 1689792   ['input_2[0][0]']             
                                                          0                                       
                                                                                                  
 tf.split (TFOpLambda)       [(None, 8),                  0         ['ConvLayers[0][0]']          
                              (None, 8),                                                          
                              (None, 8),                                                    

## Rascunhos

In [ ]:
def convlayer(hidden_ch=128,
              strides=[(1,1),(1,1)],
              norm='layer_norm2d'):
    
    
    conv2d_1x3 = tf.keras.layers.Conv2D(hidden_ch,
                                        kernel_size=(1, 3),
                                        strides=strides[0],
                                        padding='SAME',
                                        dilation_rate=(1, 1),
                                        kernel_initializer='glorot_uniform',
                                        bias_initializer='zeros')
    conv2d_3x1 = tf.keras.layers.Conv2D(hidden_ch,
                                        kernel_size=(3, 1),
                                        strides=strides[1],
                                        padding='SAME',
                                        dilation_rate=(1, 1),
                                        kernel_initializer='glorot_uniform',
                                        bias_initializer='zeros')
    
    if norm == 'layer_norm1d':
        BN_1x3 = tf.keras.layers.LayerNormalization(axis=-1)
        BN_3x1 = tf.keras.layers.LayerNormalization(axis=-1)
    elif norm == 'layer_norm2d':
        BN_1x3 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
        BN_3x1 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
    else:
        BN_1x3 = tf.keras.layers.BatchNormalization(axis=-1) # Fix axis: 2020 Apr20
        BN_3x1 = tf.keras.layers.BatchNormalization(axis=-1)
        
    forward = tf.keras.Sequential([conv2d_1x3,
                                    tf.keras.layers.ELU(),
                                    BN_1x3,
                                    conv2d_3x1,
                                    tf.keras.layers.ELU(),
                                    BN_3x1
                                    ])
    
    return forward




def denselayer(input, q=128, unit_dim=[32, 1]):

    arquitetura_densas = tf.keras.Sequential(tf.keras.layers.Dense(unit_dim[0], activation='elu'), tf.keras.layers.Dense(unit_dim[1]))
    
    y_list = [0]*q # lista vazia de tamanho 128, para guardar as 128 redes

    #input = tf.ones((120, 1, 1, 1024))
    x = tf.reshape(input, shape=[input.shape[0], q, -1])

    x_split = tf.split(x, num_or_size_splits=128, axis=1) # faz o split em 128 vetores de igual tamanho (8)

    #Dados as 128 redes
    for v in enumerate(x_split):
            y_list[v] = arquitetura_densas

    
    #concatena a lista, ficando com um emb de 128
    return tf.keras.layers.Concatenate(y_list)


def create_sequential_front_conv(input_shape=(256,32,1),
                                    emb_sz=128, # q
                                    front_hidden_ch=[128, 128, 256, 256, 512, 512, 1024, 1024],
                                    front_strides=[[(1,2), (2,1)], [(1,2), (2,1)],
                                                   [(1,2), (2,1)], [(1,2), (2,1)],
                                                   [(1,1), (2,1)], [(1,2), (2,1)],
                                                   [(1,1), (2,1)], [(1,2), (2,1)]],
                                    norm='layer_norm2d'):

    front_conv = tf.keras.Sequential(name='ConvLayers')
    if ((front_hidden_ch[-1] % emb_sz) != 0):
        front_hidden_ch[-1] = ((front_hidden_ch[-1]//emb_sz) + 1) * emb_sz

    for i in range(len(front_strides)):
        front_conv.add(convlayer(hidden_ch=front_hidden_ch[i], strides=front_strides[i], norm=norm))
    front_conv.add(tf.keras.layers.Flatten())

    return front_conv



def div_enc_layer(input, emb_sz=128, fc_unit_dim=[32,1]):
    
    div_enc = tf.keras.Sequential(name="DivEnclayer")
    div_enc.add(denselayer(input, q=emb_sz, unit_dim=fc_unit_dim))

    return div_enc




def get_fingerprinting(conv_model, div_enc):
    
    fingerprinting_model = tf.keras.Sequential(name='Fingerprinting')
    fingerprinting_model.add(conv_model)
    fingerprinting_model.add(div_enc)
    #meter aqui o dense
    fingerprinting_model.add(tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))
    
    return fingerprinting_model




conv_layer = create_sequential_front_conv()
enc_layer = div_enc_layer()

finger_model = get_fingerprinting(conv_layer, enc_layer)

y = finger_model(tf.ones(shape=(120, 256, 32, 1)))

In [7]:
def convlayer(hidden_ch=128,
              strides=[(1,1),(1,1)],
              norm='layer_norm2d'):
    
    
    conv2d_1x3 = tf.keras.layers.Conv2D(hidden_ch,
                                        kernel_size=(1, 3),
                                        strides=strides[0],
                                        padding='SAME',
                                        dilation_rate=(1, 1),
                                        kernel_initializer='glorot_uniform',
                                        bias_initializer='zeros')
    conv2d_3x1 = tf.keras.layers.Conv2D(hidden_ch,
                                        kernel_size=(3, 1),
                                        strides=strides[1],
                                        padding='SAME',
                                        dilation_rate=(1, 1),
                                        kernel_initializer='glorot_uniform',
                                        bias_initializer='zeros')
    
    if norm == 'layer_norm1d':
        BN_1x3 = tf.keras.layers.LayerNormalization(axis=-1)
        BN_3x1 = tf.keras.layers.LayerNormalization(axis=-1)
    elif norm == 'layer_norm2d':
        BN_1x3 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
        BN_3x1 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
    else:
        BN_1x3 = tf.keras.layers.BatchNormalization(axis=-1) # Fix axis: 2020 Apr20
        BN_3x1 = tf.keras.layers.BatchNormalization(axis=-1)
        
    forward = tf.keras.Sequential([conv2d_1x3,
                                    tf.keras.layers.ELU(),
                                    BN_1x3,
                                    conv2d_3x1,
                                    tf.keras.layers.ELU(),
                                    BN_3x1
                                    ])
    
    return forward


def divenclayer(input, q=128, unit_dim=[32, 1]):
    
    def arquiteturas_densas(input):
        x = tf.keras.layers.Dense(unit_dim[0], activation='elu')(input)
        x = tf.keras.layers.Dense(unit_dim[1])(x)
        return x
    
    def camadas_densas(input):
        y_list = [0]*q # lista vazia de tamanho 128, para guardar as 128 redes

        #input = tf.ones((120, 1, 1, 1024))
        x = tf.reshape(input, shape=[input.shape[0], q, -1])

        x_split = tf.split(x, num_or_size_splits=128, axis=1) # faz o split em 128 vetores de igual tamanho (8)

        #Dados as 128 redes
        for v in enumerate(x_split):
                y_list[v] = tf.keras.Sequential(arquiteturas_densas(input=v))

        return y_list
    
    #concatena a lista, ficando com um emb de 128
    return tf.keras.layers.Concatenate(camadas_densas(input=input))
    
     





    """
    #tf.split(value, num_or_size_splits, axis=0, num=None, name='split')

    x_split = tf.split(value=input, num_or_size_splits=q, name='split')

    y_list = [0]*q 

    #Dados as 128 redes, 
    for v in enumerate(x_split):
        #for k in enumerate(v):
        y_list[v] = tf.keras.Sequential(camadas_densas(v))

    
    
    
    1 camada - split, divir em 128 partes, deve sair uma lista de pedaços, vour fazer um for a partir dos dados do split, e vou fazer o y= split(x), um x_list = tf.keras
    vou dar um for em cima do negocio,, y=denseparallel (x)


    x_list = tf.keras.layers.split (x)

    y_list=[0]*128

    for v in enumerate(x_split):
        for k, enumerate(x_list):
            y[k] = denseparallel(v)
        #aloca a lista e no for já indexo a list adireto
    
        x=tf.keras.concatenado(x_list)

    
    def construct_layers():
        layers = list()
        for _ in range(q): # q: num_slices
            layers.append(tf.keras.Sequential([tf.keras.layers.Dense(unit_dim[0], activation='elu'),
                                               tf.keras.layers.Dense(unit_dim[1])]))
        return layers
            
    x = tf.reshape(x, shape=[x.shape[0], q, -1])
    
    out = list()
    for i in range(q):
        out.append(construct_layers[i](x[:, i, :]))
    
    return tf.concat(out, axis=1)
"""

In [8]:
def create_sequential_front_conv(input_shape=(256,32,1),
                                    emb_sz=128, # q
                                    front_hidden_ch=[128, 128, 256, 256, 512, 512, 1024, 1024],
                                    front_strides=[[(1,2), (2,1)], [(1,2), (2,1)],
                                                   [(1,2), (2,1)], [(1,2), (2,1)],
                                                   [(1,1), (2,1)], [(1,2), (2,1)],
                                                   [(1,1), (2,1)], [(1,2), (2,1)]],
                                    norm='layer_norm2d'):

    front_conv = tf.keras.Sequential(name='ConvLayers')
    if ((front_hidden_ch[-1] % emb_sz) != 0):
        front_hidden_ch[-1] = ((front_hidden_ch[-1]//emb_sz) + 1) * emb_sz

    for i in range(len(front_strides)):
        front_conv.add(convlayer(hidden_ch=front_hidden_ch[i], strides=front_strides[i], norm=norm))
    front_conv.add(tf.keras.layers.Flatten())

    return front_conv

def div_enc_layer(emb_sz=128, fc_unit_dim=[32,1], norm='layer_norm2d'):
    
    div_enc = tf.keras.Sequential(name="DivEnclayer")
    div_enc.add(divenclayer(q=emb_sz, unit_dim=fc_unit_dim))
    #div_enc.add(DivEncLayer(q=emb_sz, unit_dim=fc_unit_dim, norm=norm))

    return div_enc


def get_fingerprinting(conv_model, div_enc):
    
    fingerprinting_model = tf.keras.Sequential(name='Fingerprinting')
    fingerprinting_model.add(conv_model)
    fingerprinting_model.add(div_enc)
    fingerprinting_model.add(tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))
    
    return fingerprinting_model

In [ ]:
conv_layer = create_sequential_front_conv()
enc_layer = div_enc_layer()

finger_model = get_fingerprinting(conv_layer, enc_layer) 

In [7]:
y = finger_model(tf.ones(shape=(120, 256, 32, 1)))

In [8]:
finger_model.summary()

Model: "Fingerprinting"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ConvLayers (Sequential)     (120, 1024)               16897920  
                                                                 
 DivEnclayer (Sequential)    (120, 128)                41088     
                                                                 
 lambda (Lambda)             (120, 128)                0         
                                                                 
Total params: 16939008 (64.62 MB)
Trainable params: 16939008 (64.62 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import tensorflow as tf

# Função para criar o bloco de camadas densas
def create_dense_block(units: list):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(units[0], activation='elu'),
        tf.keras.layers.Dense(units[1])
    ])

# Criar o modelo
def create_model(input_shape=(256, 256, 3), num_blocks=128, block_size=8):
    model = tf.keras.Sequential()
    
    # Primeiro bloco convolucional e average pooling
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.GlobalAveragePooling2D())  # Average pooling para saída 1024
    
    # Divisão em 128 blocos
    split_layer = tf.keras.layers.Lambda(lambda x: tf.split(x, num_blocks, axis=1))
    model.add(split_layer)
    
    # Sequencial de camadas densas para cada bloco
    for _ in range(num_blocks):
        model.add(create_dense_block(units=[32,1]))
    
    # Concatenação dos resultados
    model.add(tf.keras.layers.Concatenate(axis=1))
    
    return model

# Criar o modelo
model = create_model()
# Sumário do modelo
#model.summary()

In [ ]:
class DivEncLayer(tf.keras.layers.Layer):
    """
    Multi-head projection a.k.a. 'divide and encode' layer:
        
    • The concept of 'divide and encode' was discovered  in Lai et.al.,
     'Simultaneous Feature Learning and Hash Coding with Deep Neural Networks',
      2015. https://arxiv.org/abs/1504.03410
    • It was also adopted in Gfeller et.al. 'Now Playing: Continuo-
      us low-power music recognition', 2017. https://arxiv.org/abs/1711.10958
    
    Arguments
    ---------
    q: (int) number of slices as 'slice_length = input_dim / q'
    unit_dim: [(int), (int)]
    norm: 'layer_norm1d' or 'layer_norm2d' uses 1D-layer normalization on the feature.
          'batch_norm' or else uses batch normalization. Default is 'layer_norm2d'.

    Input
    -----
    x: (B,1,1,C)
    
    Returns
    -------
    emb: (B,Q)
    
    """
    def __init__(self, q=128, unit_dim=[32, 1], norm='batch_norm'):
        super(DivEncLayer, self).__init__()

        self.q = q
        self.unit_dim = unit_dim
        self.norm = norm
        
        if norm in ['layer_norm1d', 'layer_norm2d']:
            self.BN = [tf.keras.layers.LayerNormalization(axis=-1) for i in range(q)]
        else:
            self.BN = [tf.keras.layers.BatchNormalization(axis=-1) for i in range(q)]
            
        self.split_fc_layers = self._construct_layers() 


    def build(self, input_shape):
        # Prepare output embedding variable for dynamic batch-size 
        self.slice_length = int(input_shape[-1] / self.q)

 
    def _construct_layers(self):
        layers = list()
        for i in range(self.q): # q: num_slices
            layers.append(tf.keras.Sequential([tf.keras.layers.Dense(self.unit_dim[0], activation='elu'),
                                               #self.BN[i],
                                               tf.keras.layers.Dense(self.unit_dim[1])]))
        return layers

 
    @tf.function
    def _split_encoding(self, x_slices):
        """
        Input: (B,Q,S)
        Returns: (B,Q)
        
        """
        out = list()
        for i in range(self.q):
            out.append(self.split_fc_layers[i](x_slices[:, i, :]))
        return tf.concat(out, axis=1)

    
    def call(self, x): # x: (B,1,1,2048)
        x = tf.reshape(x, shape=[x.shape[0], self.q, -1]) # (B,Q,S); Q=num_slices; S=slice length; (B,128,8 or 16)
        return self._split_encoding(x)